In [139]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important; }</style>"))
import matplotlib.pyplot as plt
font = {'size':14}
import matplotlib
matplotlib.rc('font',**font)
from datetime import datetime
from wordcloud import WordCloud, STOPWORDS


DATASETS A UTILIZAR:

In [140]:
udemy = pd.read_csv('DATASETS/udemy.csv')
#edx = pd.read_csv('DATASETS/edx.csv')
#oulad = pd.read_csv('DATASETS/oulad_student.csv')
#coursera_rev = pd.read_csv('DATASETS/Coursera_reviews.csv')
#coursera_cursos = pd.read_csv('DATASETS/coursera_data.csv')

In [152]:
edx = pd.read_csv('DATASETS/edx.csv')

In [ ]:
oulad = pd.read_csv('DATASETS/oulad_student.csv')

In [ ]:
coursera_rev = pd.read_csv('DATASETS/Coursera_reviews.csv')

In [ ]:
coursera_data = pd.read_csv('DATASETS/coursera_data.csv')

### TRANSFORMACION ###

In [141]:
# transformar a timestamp
# extraer anio y mes
udemy['published_timestamp'] = pd.to_datetime(udemy['published_timestamp'])
udemy['published_year'] = udemy['published_timestamp'].dt.year
udemy['published_month'] = udemy['published_timestamp'].dt.month

In [142]:
# convertir campos int u obj, a informacion categorica
udemy['course_id'] = udemy['course_id'].astype('category')
udemy['level'] = udemy['level'].astype('category')
udemy['subject'] = udemy['subject'].astype('category')

In [153]:
edx['course_type'] = edx['course_type'].astype('category')
edx['Level'] = edx['Level'].astype('category')
edx['subject'] = edx['subject'].astype('category')
edx['language'] = edx['language'].astype('category')

In [ ]:
coursera_data['course_Certificate_type'] = coursera_data["course_Certificate_type"].astype('category') 
coursera_data['course_difficulty'] = coursera_data["course_difficulty"].astype('category') 

In [143]:
# convertir datos bool 
diccionario = {True : 'si', False: 'no'}
udemy['isPaid'] = udemy.is_paid.map(diccionario)

In [144]:
#discretizacion udemy
#el número de intervalos y la amplitud se han calculado manualmente
# se usan las marcas de clase como etiquetas
content_bins = [0, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77]
content_labels = ['0-6.99', '7-13.99', '14-20.99', '21-27.99', '28-34.99', '35-41.99', '42-48.99', '49-55.99', '56-62.99', '63-69.99', '70-76.99']

In [145]:
duration = pd.cut(udemy.content_duration, content_bins, labels = content_labels)
udemy['rank_duration_hours'] = duration
udemy['cMark_duration'] = duration

In [146]:
def marcaClase(dato):
    mapping = {
        '0-6.99': 3.5,
        '7-13.99': 10.5,
        '14-20.99': 17.5,
        '21-27.99': 24.5,
        '28-34.99': 31.5,
        '35-41.99': 38.5,
        '42-48.99': 45.5,
        '49-55.99': 52.5,
        '56-62.99': 59.5,
        '63-69.99': 66.5,
        '70-76.99': 73.5,
        '77-83.99': 80.5,
    }
    return mapping.get(dato, dato)

udemy['cMark_duration'] = udemy['cMark_duration'].apply(marcaClase)


In [147]:
lecture_bins = [0, 60, 120, 180, 240, 300, 360, 420, 480, 540, 600, 660, 720]
lecture_labels = ['0-59.99', '60-119.99', '120-179.99', '180-239.99', '240-299.99', '300-359.99', '360-419.99', '420-479.99', '480-539.99', '540-599.99', '600-659.99', '660-719.99']

nLectures = pd.cut(udemy.num_lectures, lecture_bins, labels = lecture_labels)
udemy['rank_nLectures'] = nLectures
udemy['cMark_nLectures'] = nLectures

In [148]:
def marcaClaseLec(dato):
    mapping = {
        '0-59.99': 30.0,
        '60-119.99': 90.0,
        '120-179.99': 150.0,
        '180-239.99': 210.0,
        '240-299.99': 270.0,
        '300-359.99': 330.0,
        '360-419.99': 390.0,
        '420-479.99': 450.0,
        '480-539.99': 510.0,
        '540-599.99': 570.0,
        '600-659.99': 630.0,
        '660-719.99': 690.0,
    }
    return mapping.get(dato, dato)

udemy['cMark_nLectures'] = udemy['cMark_nLectures'].apply(marcaClaseLec)

In [149]:
#se agrega unidad de medida de tiempo 
udemy = udemy.rename(columns = {'content_duration':'total_duration_hours'})

In [150]:
# se eliminan columnas que no se usarán
del udemy['url']
del udemy['is_paid']

In [154]:
#convertír los nan  de num de inscruptos a 0
edx['n_enrolled'] = edx['n_enrolled'].str.replace(',', '').fillna(0).astype(int)

In [155]:
#extraer precio del certificado
edx[['precio2','certificate_price']] = edx.price.str.split(pat = '$', expand = True)
edx['certificate_price'] = edx['certificate_price'].str.extract(r'(\d+\.?\d*)').astype(float)
del edx['precio2']

In [156]:
del edx['course_url']
del edx['instructors']

In [ ]:
# discretizacion edx

In [163]:
edx.certificate_price.sort_values(ascending=True)

417      5.0
458      5.0
466      5.0
136      5.0
6        5.0
       ...  
594    375.0
19     399.0
842    399.0
543    399.6
157    450.0
Name: certificate_price, Length: 975, dtype: float64

In [166]:
# discretizar edx price
cert_bins = [5, 46, 87, 128, 169, 210, 251, 292, 333, 374, 415]
cert_labels = ['5-45.99', '46-86.99', '87-127.99', '128-168.99', '169-209.99', '210-250.99', '251-291.99', '292-332.99', '333-373.99', '374-415.99']

cert_price = pd.cut(edx.certificate_price, cert_bins, labels = cert_labels)
edx['rank_cert_price'] = cert_price
edx['cMark_cert_price'] = cert_price

In [167]:
def marcaClaseCert(dato):
    mapping = {
        '5-45.99': 25.5,
        '46-86.99': 66.5,
        '87-127.99': 107.5,
        '128-168.99': 148.5,
        '169-209.99': 189.5,
        '210-250.99': 230.5,
        '251-291.99': 271.5,
        '292-332.99': 312.5,
        '333-373.99': 353.5,
        '374-415.99': 394.5,
    }
    return mapping.get(dato, dato)

edx['cMark_cert_price'] = edx['cMark_cert_price'].apply(marcaClaseCert)

In [ ]:
#agrego indice para unir con otro df
coursera_rev['index'] = range(len(coursera_rev))
coursera_data['index'] = range(len(coursera_data))

In [ ]:
# fecha a formato timestamp
# extraigo anio
coursera_rev['date_reviews'] = coursera_rev['date_reviews'].apply(lambda x: datetime.strptime(x, '%b %d, %Y'))
coursera_rev['year'] = coursera_rev['date_reviews'].dt.year


In [ ]:
edx.info()

In [ ]:
coursera_rev[coursera_rev['reviews'].isna()]

In [ ]:
#join con cursos
#traigo lo que necesito para identificar al curso sobre el que se hace la resenia
columnas = ['name','institution', 'course_id']
coursera_cursos = pd.read_csv('DATASETS\Coursera_courses.csv', usecols=columnas)

In [ ]:
# se eliminan duplicados
print(coursera_cursos[coursera_cursos['name'].duplicated()])
coursera_cursos = coursera_cursos.drop_duplicates(subset = 'name')

In [ ]:
# join
coursera_rev = coursera_rev.merge(coursera_cursos, how = 'outer', on = 'course_id')

In [ ]:
#se eliminan los registros vacios para Rating
coursera_rev = coursera_rev.dropna(subset = 'rating')

In [ ]:
# convierto campo: cantidad de inscriptos
def convertir_cifra(numero):
    numero = numero.lower()
    if 'k' in numero:
        numero_sin_k = numero.replace('k', '')
        if '.' in numero_sin_k:
            return float(numero_sin_k) * 1000
        else:
            return int(numero_sin_k) * 1000
    elif 'm' in numero:
        numero_sin_m = numero.replace('m', '')
        if '.' in numero_sin_m:
            return float(numero_sin_m) * 1000000
        else:
            return int(numero_sin_m) * 1000000
    else:
        return int(numero)


In [ ]:
coursera_data['course_students_enrolled'] = coursera_data['course_students_enrolled'].apply(convertir_cifra)

In [ ]:
# se exportan los archivos ya limpios para trabajar en PowerBI

In [151]:
udemy.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL 2 analytics\repo P2\dsets\udemy.csv')

In [168]:
edx.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL 2 analytics\repo P2\dsets\edx.csv')

In [ ]:
coursera_rev.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL 2 analytics\repo P2\dsets\C_reviews.csv')

In [ ]:
coursera_data.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL 2 analytics\repo P2\dsets\C_data.csv')